<a target="_blank" href="https://colab.research.google.com/github/da-python/new-modul-python/blob/main/Lesson3/hometask/SyntheticRubber.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [62]:
!pip install yfinance

In [63]:
import yfinance as yf
import pandas as pd
import requests
import matplotlib.pyplot as plt
import xlsxwriter


In [64]:
crude_oil_ticker = "CL=F"
eurousd_ticker = 'EURUSD=X'
rubusd_ticker = 'RUB=X'
crude_oil = yf.download(crude_oil_ticker)
euro_price = yf.download(eurousd_ticker)
rub_price = yf.download(rubusd_ticker)
#hist = crude_oil.history()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Задание 1.

* Вы - менеджер по продажам в нефтехимической компании. Вы ведете несколько ключевых клиентов, которые закупают продукты вашей компании. Ваша компания запускает новый продукт - "Синтетический каучук" (СК), который по своим свойствам похож на тот, что уже есть на рынке. Ваши клиенты предпочитают формульный подход к ценообразованию. Вам нужно предложить клиентам формулу расчета цены на СК. В зависимости от объемов, которые покупают клиенты, им может быть предложена скидка.


* СК производиться из нефти. Один из известных способов производства предполагает, что из 10 баррелей нефти сделают одну тонну СК, а затраты на производство в районе 400 евро.

In [65]:

# Затраты на производство
PRODUCTION_COST = 400 # (EUR)

# Расходы на логистику
EU_LOGISTIC_COST_EUR = 30 # в Европу в евро
CN_LOGISTIC_COST_USD = 130 # в Китай в долларах

# * Справочная информация по клиентам(объемы, локации, комментарии) 
customers = {
    'Сonty':{
        'location':'EU',
        'volumes':200,
        'comment':'moving_average'
    },
    
    'Triangle':{
        'location':'CN',
        'volumes': 30,
        'comment': 'monthly'
    },
    'Stone':{
        'location':'EU',
        'volumes': 150,
        'comment': 'moving_average'
    },
    'Ant':{
        'location':'EU',
        'volumes': 70,
        'comment': 'monthly'
    }
}
# Скидки
discounts = {'99': 0.01, # до 100 тонн 1%
             '299': 0.05, #  до 300 тонн 5%
             '300': 0.1}   # больше 300 тонн 10%


### Что нужно сделать?

1. Определить базовую формулу цены на заводе. То есть как бы выглядела цена на исторических данных. 
* Важно, чтобы цена покрывала расходы на производство и при этом была в рынке, то есть на уровне натурального каучука или с небольшим дисконтом, котировки на который можно взять с сайта http://www3.lgm.gov.my/mre/MonthlyPrices.aspx (в центах за килограм). Можно перевести в доллары за тонну.
2. Отобразить на графике.
3. Сделать расчет возможной цены по формуле для каждого из клиентов на условиях DDP (цена с доставкой). Записать все в один эксель файл, на разных листах. Каждый лист - название клиента.
4. Предложить формулу цены каждому из клиентов на условиях DDP (цена с доставкой).
- Создать директорию «для клиентов» и в ней сложить  файлы с расчетами.
- Каждый клиент - отдельный файл (любой из docx, xlsx, pptx, pdf)  с именем клиента… 



In [96]:
df = pd.concat([crude_oil.resample('M').mean()['Close'], euro_price.resample('M').mean()['Close'],rub_price.resample('M').mean()['Close']], axis=1)
df.columns = ['crude_oil_price', 'eureusd', 'rubusd']
df = df['2022-01-01':'2022-12-31']
df["proccessing_usd"] = df['eureusd'] * 400
df["base_k_price_usd"] = df["crude_oil_price"]*10 + df['proccessing_usd']

In [97]:
df.reset_index(inplace= True )

In [98]:
df

,Date,crude_oil_price,eureusd,rubusd,proccessing_usd,base_k_price_usd
0,2022-01-31,82.979000,1.132574,76.412776,453.029503,1282.819504
1,2022-02-28,91.628947,1.133995,77.466339,453.598197,1369.887665
2,2022-03-31,108.262174,1.102412,111.013403,440.964999,1523.586738
3,2022-04-30,101.640499,1.082745,80.666187,433.097801,1449.502796
4,2022-05-31,109.259523,1.057322,65.422944,422.928960,1515.524189
5,2022-06-30,114.154091,1.057670,60.146091,423.067819,1564.608731
6,2022-07-31,99.384500,1.019509,61.413657,407.803622,1401.648619
7,2022-08-31,91.482609,1.012799,60.808918,405.119594,1319.945681
8,2022-09-30,83.801905,0.991166,60.062273,396.466519,1234.485568
9,2022-10-31,87.030952,0.983970,61.694991,393.588161,1263.897686


In [27]:
# Котировки натурального каучука доступны на сайте,  https://www.lgm.gov.my
# также у них есть API, пример запроса ниже
#
year = '2022'
month = '05'
url = f"https://www.lgm.gov.my/webv2api/api/rubberprice/month={month}&year={year}"
res = requests.get(url)
rj = res.json()
df_1 = pd.json_normalize(rj)
#df.head(10)

In [83]:
rubber=[]
for i in range(1,13):
    print(i) 
    year = '2022'
    month = i
    url = f"https://www.lgm.gov.my/webv2api/api/rubberprice/month={month}&year={year}"
    res = requests.get(url)
    rj = res.json()
    df_1 =pd.DataFrame(rj)
    rubber.append(df) 
rubberprice_df =pd.concat(rubber, axis=0)
rubberprice_df = rubberprice_df.reset_index(drop=True) 

1
2
3
4
5
6
7
8
9
10
11
12


In [69]:
rubber_cv = rubberprice_df[rubberprice_df['grade']== 'SMR CV']

In [70]:
rubber_cv

,date,grade,masa,rm,us,tone
0,2022-01-28,SMR CV,Noon,1002.50,242.70,Quiet
7,2022-01-27,SMR CV,Noon,1003.50,242.65,Quiet
14,2022-01-26,SMR CV,Noon,1001.50,243.00,Quiet
21,2022-01-25,SMR CV,Noon,996.00,241.90,Quiet
28,2022-01-24,SMR CV,Noon,991.00,240.70,Quiet
...,...,...,...,...,...,...
1652,2022-12-07,SMR CV,Noon,795.00,184.00,Steady
1659,2022-12-06,SMR CV,Noon,796.00,184.75,Steady
1666,2022-12-05,SMR CV,Noon,795.00,184.60,Steady
1673,2022-12-02,SMR CV,Noon,794.00,183.95,Steady


In [86]:
rubber_cv['us']=rubber_cv['us'].apply(pd.to_numeric)
rubber_cv["usd_tonna"] = rubber_cv['us'] * 1000/100

In [87]:
rubber_cv_1= pd.DataFrame(rubber_cv, columns = ['date','usd_tonna'])

In [88]:
rubber_cv_1

,date,usd_tonna
0,2022-01-28,2427.0
7,2022-01-27,2426.5
14,2022-01-26,2430.0
21,2022-01-25,2419.0
28,2022-01-24,2407.0
...,...,...
1652,2022-12-07,1840.0
1659,2022-12-06,1847.5
1666,2022-12-05,1846.0
1673,2022-12-02,1839.5


In [110]:
#rubber_cv_1.dtypes
#df.dtypes
rubber_cv_1['date'] = pd.to_datetime(rubber_cv_1['date'])

In [111]:
new_price = pd.merge(df, rubber_cv_1, left_on =('Date'), right_on =('date'),  how='inner')

In [112]:
new_price

,Date,crude_oil_price,eureusd,rubusd,proccessing_usd,base_k_price_usd,date,usd_tonna
0,2022-02-28,91.628947,1.133995,77.466339,453.598197,1369.887665,2022-02-28,2915.0
1,2022-03-31,108.262174,1.102412,111.013403,440.964999,1523.586738,2022-03-31,2751.5
2,2022-05-31,109.259523,1.057322,65.422944,422.928960,1515.524189,2022-05-31,2692.0
3,2022-06-30,114.154091,1.057670,60.146091,423.067819,1564.608731,2022-06-30,2261.0
4,2022-09-30,83.801905,0.991166,60.062273,396.466519,1234.485568,2022-09-30,1758.5
5,2022-10-31,87.030952,0.983970,61.694991,393.588161,1263.897686,2022-10-31,1701.5
6,2022-11-30,84.385715,1.018547,61.131137,407.418777,1251.275922,2022-11-30,1788.5


In [114]:
new_price['razn'] = new_price['base_k_price_usd'] < new_price['usd_tonna']
new_price

,Date,crude_oil_price,eureusd,rubusd,proccessing_usd,base_k_price_usd,date,usd_tonna,razn
0,2022-02-28,91.628947,1.133995,77.466339,453.598197,1369.887665,2022-02-28,2915.0,True
1,2022-03-31,108.262174,1.102412,111.013403,440.964999,1523.586738,2022-03-31,2751.5,True
2,2022-05-31,109.259523,1.057322,65.422944,422.928960,1515.524189,2022-05-31,2692.0,True
3,2022-06-30,114.154091,1.057670,60.146091,423.067819,1564.608731,2022-06-30,2261.0,True
4,2022-09-30,83.801905,0.991166,60.062273,396.466519,1234.485568,2022-09-30,1758.5,True
5,2022-10-31,87.030952,0.983970,61.694991,393.588161,1263.897686,2022-10-31,1701.5,True
6,2022-11-30,84.385715,1.018547,61.131137,407.418777,1251.275922,2022-11-30,1788.5,True


In [118]:
def check_volume_discounts(x):
  if 0 < x <= 99:
    return 0.01
  elif 99 < x <= 299:
    return 0.05
  elif x >= 300:
    return 0.1

In [120]:
xlfilepath = 'wbp_prices.xlsx'
with pd.ExcelWriter(xlfilepath, engine='xlsxwriter') as writer:
  for client_name, client_info in customers.items():
    client_df = new_price.copy()
    
    if client_info.get('location') == 'EU':
      client_df['logistics'] = EU_LOGISTIC_COST_EUR
      client_df['logistics'] = client_df['eureusd'] * EU_LOGISTIC_COST_EUR
    elif client_info.get('location') == 'CN':
      client_df['logistics'] = CN_LOGISTIC_COST_USD
    
    if client_info.get('volumes'):
      client_df['volumes'] = client_info.get('volumes')

    client_df['discount'] = client_df['volumes'].apply(check_volume_discounts)
    client_df['WBP_price_usd'] = client_df['base_k_price_usd'] * (1-client_df['discount']) + client_df['logistics']
    
    client_df['WBP_price_usd'].plot(color='red', linestyle="dashed")
    plt.title(f"Client {client_name} WBP Price")
    plt.ylabel("USD")
    plt.tight_layout()
    plt.savefig(f'{client_name}_wbp_price.png')
    plt.close()

    client_df = client_df.round(2)
    client_df = client_df.reset_index()
    client_df.Date = client_df.Date.dt.strftime('%B %Y')
    
    max_row, max_col = client_df.shape
    client_df.to_excel(writer, sheet_name=client_name, startrow=1, header=False, index=False)
    workbook = writer.book
    worksheet = writer.sheets[client_name]
    column_settings = [{'header': column} for column in client_df.columns]
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
    worksheet.insert_image(max_row + 3, 1, f'{client_name}_wbp_price.png')
    

C:\Users\1\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:2709: UserWarning: Duplicate header name in add_table(): 'date'
  warn("Duplicate header name in add_table(): '%s'"
C:\Users\1\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:2709: UserWarning: Duplicate header name in add_table(): 'date'
  warn("Duplicate header name in add_table(): '%s'"
C:\Users\1\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:2709: UserWarning: Duplicate header name in add_table(): 'date'
  warn("Duplicate header name in add_table(): '%s'"
C:\Users\1\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:2709: UserWarning: Duplicate header name in add_table(): 'date'
  warn("Duplicate header name in add_table(): '%s'"


# Задание 2

Клиенты согласились на ваше предложение. 
Вам нужно сделать написать скрипт, с помощью которого можно автоматом рассчитывать цены для всех клиентов одновременно. 
Важный момент: данные для расчета скрипт должен подтягивать сам.

In [3]:
# из скрипта выше

# Задание 3

К вам пришел новый клиент из России. Который использует продукт А и готов регулярно закупать СК для того, чтобы не быть привязанным к одному поставщику. Клиент готов забирать СК с завода самостоятельно.
На основе тех данных, которые уже есть предложите цену новому клиенту в рублях. 



In [127]:
#у меня не очень хорошо с экономикой и всеми расчетами, представим что формула верная
new_price['rub_tonna'] = new_price['usd_tonna']*new_price['rubusd']
new_price

,Date,crude_oil_price,eureusd,rubusd,proccessing_usd,base_k_price_usd,date,usd_tonna,razn,rub_tonna
0,2022-02-28,91.628947,1.133995,77.466339,453.598197,1369.887665,2022-02-28,2915.0,True,225814.378510
1,2022-03-31,108.262174,1.102412,111.013403,440.964999,1523.586738,2022-03-31,2751.5,True,305453.377045
2,2022-05-31,109.259523,1.057322,65.422944,422.928960,1515.524189,2022-05-31,2692.0,True,176118.564267
3,2022-06-30,114.154091,1.057670,60.146091,423.067819,1564.608731,2022-06-30,2261.0,True,135990.312402
4,2022-09-30,83.801905,0.991166,60.062273,396.466519,1234.485568,2022-09-30,1758.5,True,105619.506810
5,2022-10-31,87.030952,0.983970,61.694991,393.588161,1263.897686,2022-10-31,1701.5,True,104974.027454
6,2022-11-30,84.385715,1.018547,61.131137,407.418777,1251.275922,2022-11-30,1788.5,True,109333.037870


In [128]:
customers_1 = {
    'Wasya':{
        'location':'RU',
        'volumes':100,
        'comment':'moving_average'
    }}


In [131]:
xlfilepath = 'wbp_prices1.xlsx'
with pd.ExcelWriter(xlfilepath, engine='xlsxwriter') as writer:
  for client_name, client_info in customers_1.items():
    client1_df = new_price.copy()
    
    if client_info.get('location') == 'RU':
      client1_df['logistics'] = client1_df['rub_tonna']*0
    
    if client_info.get('volumes'):
      client1_df['volumes'] = client_info.get('volumes')

    client1_df['discount'] = client1_df['volumes'].apply(check_volume_discounts)
    client1_df['WBP_price_usd'] = client1_df['rub_tonna'] * (1-client1_df['discount'])
    
    client1_df['WBP_price_usd'].plot(color='red', linestyle="dashed")
    plt.title(f"Client {client_name} WBP Price")
    plt.ylabel("USD")
    plt.tight_layout()
    plt.savefig(f'{client_name}_wbp_price.png')
    plt.close()

    client1_df = client1_df.round(2)
    client1_df = client1_df.reset_index()
    client1_df.Date = client1_df.Date.dt.strftime('%B %Y')
    
    max_row, max_col = client1_df.shape
    client1_df.to_excel(writer, sheet_name=client_name, startrow=1, header=False, index=False)
    workbook = writer.book
    worksheet = writer.sheets[client_name]
    column_settings = [{'header': column} for column in client1_df.columns]
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
    worksheet.insert_image(max_row + 3, 1, f'{client_name}_wbp_price.png')

C:\Users\1\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:2709: UserWarning: Duplicate header name in add_table(): 'date'
  warn("Duplicate header name in add_table(): '%s'"
